## **IMPORT**

In [172]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Biblioteca para fazer a PADRONIZAÇÃO
from sklearn.preprocessing import StandardScaler

# Biblioteca para fazer a NORMALIZAÇÃO
from sklearn.preprocessing import MinMaxScaler

# Bilioteca para separação de dados em treino e teste
from sklearn.model_selection import train_test_split

from sklearn.neural_network import MLPClassifier

# Biblioteca para calcular a acuracia do modelo
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import roc_auc_score

import lime
import lime.lime_tabular

from anchor import utils
from anchor import anchor_tabular

from imblearn.over_sampling import ADASYN
from collections import Counter


## **CARREGA DADOS**

In [384]:
data = pd.read_csv('../Base de Dados/Dados_49_V.csv', na_values='?')

print(data.shape)
data.head()

(2185, 50)


,regiao,exploracao,criacao,n_ordenha,t_ordenha,inseminacao_2cat,raca_bovinos,total_animais_prop,ov_capr,eq,...,como_resfr,leite_granel,queijo_mant,final_queijo_mant,consome_leite_cru,assist_vet,tipo_assist,aguadas_bebedouro,classificacao_propriedade,resul_prop
0,6,2,3,1,1,0,2,15,0,0,...,-1,0,0,-1,1,0,-1,0,1,0
1,6,2,2,2,3,0,2,142,0,1,...,1,1,0,-1,1,0,-1,0,1,0
2,6,2,2,2,2,0,2,40,0,1,...,-1,0,0,-1,1,0,-1,0,1,0
3,6,2,2,2,1,0,2,68,0,0,...,-1,1,0,-1,1,0,-1,0,1,0
4,6,2,2,1,1,0,2,38,0,0,...,-1,0,1,2,1,0,-1,0,1,0


## **ANÁLISE DE DADOS**

In [ ]:
data['resul_prop'].value_counts().plot(kind='bar', title='Distribuição', figsize=(10, 6), rot=0)
plt.show()

print(data['resul_prop'].value_counts(normalize=True))
print(data['resul_prop'].value_counts())

#data.describe()

#data['teste_bruc'].hist()

## **TRATAMENTO DE DADOS**

In [ ]:
# Gerando um DataFrame das variáveis preditoras originais SEM a variável TARGET
VAR_PREDITORAS_ORIG = data.drop('resul_prop', axis = 1)

# Gerando um DataFrame somente da variavel TARGET
VAR_TARGET = data['resul_prop']

### **NORMALIZAÇÃO**

In [ ]:
# Criando um objeto para NORMALIZAR os dados
obj_normalizacao = MinMaxScaler().fit(VAR_PREDITORAS_ORIG)

# Aplicando a NORMALIZAÇÃO
VAR_PREDITORAS_NORM = obj_normalizacao.transform(VAR_PREDITORAS_ORIG)

# Transformando para DataFrame e nomeando as colunas
VAR_PREDITORAS_NORM = pd.DataFrame(VAR_PREDITORAS_NORM)

VAR_PREDITORAS_NORM.columns = data.drop('resul_prop', axis = 1).columns

VAR_PREDITORAS_NORM

plt.figure(figsize=(20,6))
ax = sns.boxplot(data=VAR_PREDITORAS_NORM)

### **PADRONIZAÇÃO**

In [ ]:
# Criando um objeto para PADRONIZAR os dados
obj_padronizacao = StandardScaler().fit(VAR_PREDITORAS_ORIG)

# Aplicando a PADRONIZAÇÃO
VAR_PREDITORAS_PRAD = obj_padronizacao.transform(VAR_PREDITORAS_ORIG)

# Transformando para DataFrame e nomeando as colunas
VAR_PREDITORAS_PRAD = pd.DataFrame(VAR_PREDITORAS_PRAD)

VAR_PREDITORAS_PRAD.columns = data.drop('resul_prop', axis = 1).columns

plt.figure(figsize=(20,6))
ax = sns.boxplot(data=VAR_PREDITORAS_PRAD)

## **AGRUPAMENTO**

In [169]:
DATA_NORM = VAR_PREDITORAS_NORM.assign(resul_prop=VAR_TARGET)
DATA_NORM_NEG = VAR_PREDITORAS_NORM.assign(resul_prop=VAR_TARGET).query('resul_prop == 0').reset_index(drop=True)
DATA_NORM_POS = VAR_PREDITORAS_NORM.assign(resul_prop=VAR_TARGET).query('resul_prop == 1').reset_index(drop=True)

kmeans = KMeans(n_clusters=4, random_state=0, n_init="auto").fit(DATA_NORM_NEG.drop('resul_prop', axis = 1))
#print(kmeans.labels_)

#print(pd.DataFrame(kmeans.labels_))
DATA_NORM_NEG = DATA_NORM_NEG.assign(group=pd.DataFrame(kmeans.labels_))

DATA_NORM_NEG_GROUP0 = DATA_NORM_NEG.query('group == 0')
#print(DATA_NORM_NEG_GROUP0.shape)

DATA_NORM_NEG_GROUP1 = DATA_NORM_NEG.query('group == 1')
#print(DATA_NORM_NEG_GROUP1.shape)

DATA_NORM_NEG_GROUP2 = DATA_NORM_NEG.query('group == 2')
#print(DATA_NORM_NEG_GROUP2.shape)

DATA_NORM_NEG_GROUP3 = DATA_NORM_NEG.query('group == 3')
#print(DATA_NORM_NEG_GROUP3.shape)

DATA_NORM_NEG_GROUP0 = DATA_NORM_NEG_GROUP0.sample(frac=1).drop('group', axis = 1).reset_index(drop=True)
DATA_NORM_NEG_GROUP1 = DATA_NORM_NEG_GROUP1.sample(frac=1).drop('group', axis = 1).reset_index(drop=True)
DATA_NORM_NEG_GROUP2 = DATA_NORM_NEG_GROUP2.sample(frac=1).drop('group', axis = 1).reset_index(drop=True)
DATA_NORM_NEG_GROUP3 = DATA_NORM_NEG_GROUP3.sample(frac=1).drop('group', axis = 1).reset_index(drop=True)

## **RNA**

In [214]:
CLASSE_NEG = pd.concat([DATA_NORM_NEG_GROUP0.iloc[0:30,:], DATA_NORM_NEG_GROUP1.iloc[0:30,:],
    DATA_NORM_NEG_GROUP2.iloc[0:30,:],DATA_NORM_NEG_GROUP3.iloc[0:30,:]], ignore_index = True)

DATA_NORM_POS = DATA_NORM_POS.sample(frac=1).reset_index(drop=True)
CLASSE_POS = pd.concat([DATA_NORM_POS.iloc[0:60,:],DATA_NORM_POS.iloc[0:60,:]], ignore_index = True)

CONJ_TRAIN = pd.concat([CLASSE_NEG,CLASSE_POS]).sample(frac=1).reset_index(drop=True)


X_TRAIN = CONJ_TRAIN.drop('resul_prop', axis = 1)
Y_TRAIN = CONJ_TRAIN['resul_prop']

rna = MLPClassifier(random_state=1, max_iter=10000, hidden_layer_sizes = (30)).fit(X_TRAIN, Y_TRAIN)





array([[0.57215889, 0.42784111]])

In [234]:
print(rna.predict_proba(DATA_NORM_POS.iloc[[63]].drop('resul_prop', axis = 1)))

rna.predict(DATA_NORM_POS.iloc[[62]].drop('resul_prop', axis = 1))

[[0.57215889 0.42784111]]


array([0], dtype=int64)

In [208]:
DATA_NORM_POS.iloc[[31]].drop('resul_prop', axis = 1)

,regiao,exploracao,criacao,n_ordenha,t_ordenha,inseminacao_2cat,raca_bovinos,total_animais_prop,ov_capr,eq,...,resfr_leite,como_resfr,leite_granel,queijo_mant,final_queijo_mant,consome_leite_cru,assist_vet,tipo_assist,aguadas_bebedouro,classificacao_propriedade
31,0.666667,0.5,1.0,0.666667,0.5,0.5,1.0,0.002203,0.0,0.0,...,0.5,0.0,0.5,1.0,0.666667,1.0,0.5,0.0,0.5,0.0
